# Tool 1 - Entity to Candidate Mapper (Parallel Multi-Agent Pattern)

**Status:** ✅ Ready for Databricks | **LLM Cost:** ~$0.003 per run | **Performance:** ~10s

**Pattern:** 2 Pydantic AI agents running in parallel via `asyncio.gather`

**Showcase:** Parallel multi-agent execution - both agents run simultaneously, results merged with consistency validation.

**Key Features:**
- 2 specialized agents: `ranking_agent` (top 10 candidates) + `mapping_agent` (1:1 entity mappings)
- Parallel execution with `asyncio.gather` (both agents run simultaneously)
- Consistency check validates overlap between ranked and mapped candidates (target: >70%)
- Expected performance: ~10s for parallel execution

**TODO:**
- [ ] Add retry logic for LLM failures (timeout, rate limit)
- [ ] Cache ranking results for repeated entity sets
- [ ] Test with 20+ entities (current test: 5-10)
- [ ] Add metadata quality score (completeness, freshness)

**IDEA:**
- Consider hybrid: embedding similarity for initial filtering + LLM for final ranking
- Add configurable top-N parameter (currently hardcoded to 10)
- Export consistency report as separate artifact

In [ ]:
# Install dependencies
%pip install pydantic-ai>=0.0.49 pydantic>=2.8.0

In [ ]:
# Restart Python kernel to use new packages
dbutils.library.restartPython()  # type: ignore

In [ ]:
import asyncio
import json
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent  # type: ignore

In [ ]:
# Configure Azure OpenAI from Databricks secrets
AZURE_ENDPOINT = dbutils.secrets.get(scope="mcop", key="azure-openai-endpoint")  # type: ignore
AZURE_API_KEY = dbutils.secrets.get(scope="mcop", key="azure-openai-api-key")  # type: ignore
DEPLOYMENT_NAME = dbutils.secrets.get(scope="mcop", key="azure-openai-deployment-name")  # type: ignore

# Set environment variables for Pydantic AI
os.environ["OPENAI_BASE_URL"] = AZURE_ENDPOINT
os.environ["OPENAI_API_KEY"] = AZURE_API_KEY

MODEL_NAME = f"openai:{DEPLOYMENT_NAME}"
print(f"✅ Configured model: {MODEL_NAME}")

In [ ]:
# Pydantic schemas
class CandidateRank(BaseModel):
    """Ranked candidate with score."""
    entity_id: str = Field(description="Entity ID")
    rank: int = Field(description="Rank position (1=best)")
    relevance_score: float = Field(description="Relevance 0-1")
    justification: str = Field(description="Why this candidate is relevant")

class CandidateRanking(BaseModel):
    """Top candidates ranked by relevance."""
    top_candidates: list[CandidateRank] = Field(description="Top 10 candidates")

class EntityMapping(BaseModel):
    """Entity mapped to best candidate."""
    entity_name: str = Field(description="Business entity name")
    matched_candidate_id: str = Field(description="Mapped candidate ID")
    confidence: float = Field(description="Mapping confidence 0-1")
    reasoning: str = Field(description="Mapping rationale")

class MappingSuggestions(BaseModel):
    """All entity-to-candidate mappings."""
    mappings: list[EntityMapping] = Field(description="Entity mappings")

In [ ]:
# Create 2 specialized agents
ranking_agent = Agent(
    MODEL_NAME,
    result_type=CandidateRanking,
    system_prompt="""You are a data catalog expert.

Rank candidates by relevance to business entities:
- Consider semantic similarity (name, description)
- Evaluate data quality signals (completeness, freshness)
- Prioritize candidates with metadata (owner, lineage, tags)
- Top 10 most relevant candidates

Return structured ranking with scores and justifications."""
)

mapping_agent = Agent(
    MODEL_NAME,
    result_type=MappingSuggestions,
    system_prompt="""You are a metadata mapping specialist.

Map each business entity to the BEST matching candidate:
- 1:1 mapping (one entity → one candidate)
- High confidence mappings (>0.7) preferred
- Consider semantic alignment and context
- Provide clear reasoning for each mapping

Return structured mappings with confidence scores."""
)

print("✅ Ranking and mapping agents created")

In [ ]:
async def map_entities_to_candidates(entities: list[dict], metadata: dict) -> dict:
    """Parallel multi-agent entity mapping.

    Args:
        entities: Business entities from Tool 0
        metadata: Technical metadata (Collibra/Unity Catalog)

    Returns:
        Mapping results with consistency check
    """
    # Prepare prompts for both agents
    entities_str = json.dumps(entities, indent=2)
    metadata_sample = json.dumps(list(metadata.items())[:20], indent=2)  # Top 20 to avoid overflow

    ranking_prompt = f"""Rank these candidates for the business entities:

Entities:
{entities_str}

Available Metadata:
{metadata_sample}

Return top 10 candidates ranked by relevance."""

    mapping_prompt = f"""Map each business entity to the best matching candidate:

Entities:
{entities_str}

Available Metadata:
{metadata_sample}

Return 1:1 mappings with confidence scores."""

    # **KEY PATTERN: Parallel execution with asyncio.gather**
    ranking_task = asyncio.create_task(ranking_agent.run(ranking_prompt))
    mapping_task = asyncio.create_task(mapping_agent.run(mapping_prompt))

    # Wait for both agents to complete (runs in parallel)
    ranking_result, mapping_result = await asyncio.gather(
        ranking_task,
        mapping_task
    )

    ranked = ranking_result.data
    mapped = mapping_result.data

    # Consistency check: validate overlap between ranking and mapping
    ranked_ids = {c.entity_id for c in ranked.top_candidates}
    mapped_ids = {m.matched_candidate_id for m in mapped.mappings}

    overlap = ranked_ids & mapped_ids
    consistency_ratio = len(overlap) / len(mapped_ids) if mapped_ids else 0.0

    return {
        "rankings": [c.model_dump() for c in ranked.top_candidates],
        "mappings": [m.model_dump() for m in mapped.mappings],
        "consistency_check": {
            "overlap_count": len(overlap),
            "total_mappings": len(mapped_ids),
            "consistency_ratio": consistency_ratio,
            "status": "PASS" if consistency_ratio > 0.7 else "WARN"
        }
    }

print("✅ Parallel mapping function defined")

In [ ]:
# Load input data from DBFS
tool0_path = "/dbfs/FileStore/mcop/tool0_samples/sample_business_request.json"
metadata_path = "/dbfs/FileStore/mcop/metadata/BA-BS_Datamarts_metadata.json"

with open(tool0_path, "r") as f:
    tool0_data = json.load(f)

with open(metadata_path, "r") as f:
    metadata = json.load(f)

entities = tool0_data.get("entities", [])
print(f"✅ Loaded {len(entities)} entities from Tool 0")
print(f"✅ Loaded {len(metadata)} metadata items")

In [ ]:
# Run parallel mapping
result = await map_entities_to_candidates(entities, metadata)

print(f"\n✅ Parallel mapping complete")
print(f"   Rankings: {len(result['rankings'])}")
print(f"   Mappings: {len(result['mappings'])}")
print(f"   Consistency: {result['consistency_check']['consistency_ratio']:.1%} ({result['consistency_check']['status']})")

In [ ]:
# Save results to DBFS
output_path = "/dbfs/FileStore/mcop/tool1/filtered_dataset.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(result, f, indent=2)

print(f"✅ Results saved: {output_path}")

In [ ]:
# Display sample results with consistency check
print("\n" + "="*80)
print("PARALLEL MULTI-AGENT MAPPING RESULTS")
print("="*80)

print(f"\n🏆 Top 5 Ranked Candidates:")
for candidate in result['rankings'][:5]:
    print(f"   {candidate['rank']}. {candidate['entity_id']} (score: {candidate['relevance_score']:.2f})")
    print(f"      → {candidate['justification'][:80]}...")

print(f"\n🔗 Sample Mappings (top 5):")
for mapping in result['mappings'][:5]:
    print(f"   {mapping['entity_name']} → {mapping['matched_candidate_id']}")
    print(f"      Confidence: {mapping['confidence']:.1%}")
    print(f"      Reasoning: {mapping['reasoning'][:80]}...")

print(f"\n✅ Consistency Check:")
print(f"   Overlap: {result['consistency_check']['overlap_count']} / {result['consistency_check']['total_mappings']}")
print(f"   Ratio: {result['consistency_check']['consistency_ratio']:.1%}")
print(f"   Status: {result['consistency_check']['status']}")
print(f"   Expected: >70% for good quality (ranking and mapping agree)")

print("="*80)